In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.metrics import *
#from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.neural_network import MLPClassifier

In [18]:
training_file = 'C2T1_Train.csv'
testing_file = 'C2T1_Test.csv'
mapping_path = "C2T1_IDs_mapping.csv"

In [19]:
train = pd.read_csv(training_file)
test = pd.read_csv(testing_file)
mapping = pd.read_csv(mapping_path)

print(len(train))
print(len(test))
print(len(mapping))

90766
11000
67


In [21]:
mapping

,admission_type_id,description
0,1,Emergency
1,2,Urgent
2,3,Elective
3,4,Newborn
4,5,Not Available
...,...,...
62,22,Transfer from hospital inpt/same fac reslt in...
63,23,Born inside this hospital
64,24,Born outside this hospital
65,25,Transfer from Ambulatory Surgery Center


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90766 entries, 0 to 90765
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   encounter_id2             90766 non-null  int64 
 1   patient_nbr2              90766 non-null  int64 
 2   race                      90766 non-null  object
 3   gender                    90766 non-null  object
 4   age                       90766 non-null  object
 5   weight                    90766 non-null  object
 6   admission_type_id         90766 non-null  int64 
 7   discharge_disposition_id  90766 non-null  int64 
 8   admission_source_id       90766 non-null  int64 
 9   time_in_hospital          90766 non-null  int64 
 10  payer_code                90766 non-null  object
 11  medical_specialty         90766 non-null  object
 12  num_lab_procedures        90766 non-null  int64 
 13  num_procedures            90766 non-null  int64 
 14  num_medications       

In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   encounter_id              11000 non-null  int64  
 1   patient_nbr               11000 non-null  int64  
 2   race                      11000 non-null  object 
 3   gender                    11000 non-null  object 
 4   age                       11000 non-null  object 
 5   weight                    11000 non-null  object 
 6   admission_type_id         11000 non-null  int64  
 7   discharge_disposition_id  11000 non-null  int64  
 8   admission_source_id       11000 non-null  int64  
 9   time_in_hospital          11000 non-null  int64  
 10  payer_code                11000 non-null  object 
 11  medical_specialty         11000 non-null  object 
 12  num_lab_procedures        11000 non-null  int64  
 13  num_procedures            11000 non-null  int64  
 14  num_me

In [20]:
mapping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   admission_type_id  65 non-null     object
 1   description        62 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [10]:
train.describe()

,encounter_id2,patient_nbr2,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,9.076600e+04,9.076600e+04,90766.000000,90766.000000,90766.000000,90766.000000,90766.000000,90766.000000,90766.000000,90766.000000,90766.000000,90766.000000,90766.000000
mean,1.628244e+08,5.291868e+07,2.025406,3.775092,5.847267,4.408942,42.848622,1.341868,15.912886,0.347090,0.189730,0.624628,7.379371
std,1.083792e+08,3.954496e+07,1.454453,5.359248,4.147569,2.995873,19.796106,1.704650,8.122734,1.194099,0.930657,1.256617,1.950545
min,5.283000e+03,5.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,7.794063e+07,2.220677e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.402534e+08,4.361397e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,2.421968e+08,8.681160e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,4.438600e+08,1.895025e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,40.000000,76.000000,21.000000,16.000000


In [11]:
train.isnull().sum()

encounter_id2                   0
patient_nbr2                    0
race                            0
gender                          0
age                             0
weight                          0
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                      0
medical_specialty               0
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                          0
diag_2                          0
diag_3                          0
number_diagnoses                0
max_glu_serum               85581
A1Cresult                   75927
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

In [12]:
test.isnull().sum()

encounter_id                    0
patient_nbr                     0
race                            0
gender                          0
age                             0
weight                          0
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                      0
medical_specialty               0
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                          0
diag_2                          0
diag_3                          0
number_diagnoses                0
max_glu_serum               10839
A1Cresult                    8821
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

In [14]:
train['readmitted'].value_counts()

readmitted
NO     49361
>30    31232
<30    10173
Name: count, dtype: int64